# Micro-Agent: Complete Walkthrough

**A notebook-style tutorial from install to production use.**

This notebook walks through the entire `micro-agent` (`ma-loop`) system — what it is, how it works,
and how to use it — with real runnable examples at every step.

---

## Further Reading

| Topic | Doc |
|---|---|
| TypeScript / JavaScript projects | [`docs/tutorials/typescript-javascript.md`](../tutorials/typescript-javascript.md) |
| Python projects | [`docs/tutorials/python.md`](../tutorials/python.md) |
| Rust projects | [`docs/tutorials/rust.md`](../tutorials/rust.md) |
| Model configuration (which LLM for each agent) | [`docs/tutorials/model-configuration.md`](../tutorials/model-configuration.md) |
| Simple Mode + Escalation user guide | [`specs/002-simple-escalation/quickstart.md`](../../specs/002-simple-escalation/quickstart.md) |
| CLI flags contract | [`specs/002-simple-escalation/contracts/cli-interface.md`](../../specs/002-simple-escalation/contracts/cli-interface.md) |

---

## Part 1: What is Micro-Agent?

Micro-agent is an **autonomous code-fixing loop** (the *Ralph Loop*) that:

1. Takes a broken file (or a description of what to build)
2. Runs your test suite to see what's failing
3. Uses LLMs to generate a fix
4. Runs your tests again
5. Repeats until tests pass — or budget runs out

### Two modes

```
Simple Mode (default)
─────────────────────
  Artisan → Tests
  Artisan → Tests
  Artisan → Tests   ← exits as soon as tests pass
  (up to N iterations)

Full Mode (triggered automatically or via --full)
────────────────────────────────────────────────
  Librarian → Artisan → Critic → Tests
  Librarian → Artisan → Critic → Tests
  (up to remaining budget)
```

### The three agents

| Agent | Model | Role |
|---|---|---|
| **Librarian** | Gemini 2.0 Pro | Reads your codebase, builds a dependency graph, ranks relevant files, produces a context summary for Artisan |
| **Artisan** | Claude Sonnet | Receives the context summary + failing test output, writes the fix |
| **Critic** | GPT-4.1 Mini | Reviews the Artisan's code change before tests run — approves or rejects |

> **Simple mode skips Librarian and Critic.** Just Artisan + Tests.
> Cheap, fast, and correct for most simple bugs.

## Part 2: Installation


In [ ]:
# Clone and install
!git clone https://github.com/gyasis/micro-agent.git
%cd micro-agent
!npm install
!npm run build

In [ ]:
# Verify the CLI is available
!npx ma-loop --help

**Expected output:**
```
Usage: ma-loop [options] [command]

Micro Agent - AI-powered code iteration loop

Options:
  -V, --version   output the version number
  -h, --help      display help for command

Commands:
  run [options] <target>   Run the Ralph Loop on a target file or objective
  status                   Show current loop status
  ...
```

## Part 3: API Keys

Each agent uses a different LLM provider. You need keys for all three.

In [ ]:
# Create a .env file with your keys
env_content = """# Anthropic — used by Artisan (Claude Sonnet)
ANTHROPIC_API_KEY=sk-ant-...

# Google — used by Librarian (Gemini 2.0 Pro)
GOOGLE_API_KEY=AIza...

# OpenAI — used by Critic (GPT-4.1 Mini)
OPENAI_API_KEY=sk-...
"""

with open('.env', 'w') as f:
    f.write(env_content)

print("✓ .env written — edit it with your actual keys before running")

> See [`docs/tutorials/model-configuration.md`](../tutorials/model-configuration.md) for how to swap providers,
> use local models (Ollama), or configure per-agent temperature.

## Part 4: Your First Bug Fix — Simple Mode

Let's create a broken TypeScript file and watch `ma-loop` fix it.

In [ ]:
import os

# Create a minimal project directory
os.makedirs('demo/src', exist_ok=True)

# The broken implementation
broken_code = '''
/**
 * Math utilities with a deliberate bug.
 * The multiply function uses + instead of *.
 */
export function add(a: number, b: number): number {
  return a + b;
}

export function multiply(a: number, b: number): number {
  return a + b;  // BUG: should be a * b
}

export function subtract(a: number, b: number): number {
  return a - b;
}
'''

with open('demo/src/math.ts', 'w') as f:
    f.write(broken_code)

print("✓ Broken file created: demo/src/math.ts")
print("  Bug: multiply() uses + instead of *")

In [ ]:
# The test that will fail
test_code = '''
import { describe, it, expect } from "vitest";
import { add, multiply, subtract } from "../src/math";

describe("Math utilities", () => {
  it("add works", () => {
    expect(add(3, 4)).toBe(7);
  });

  it("multiply works", () => {
    expect(multiply(3, 4)).toBe(12);  // will fail: 3 + 4 = 7, not 12
  });

  it("subtract works", () => {
    expect(subtract(10, 3)).toBe(7);
  });
});
'''

os.makedirs('demo/tests', exist_ok=True)
with open('demo/tests/math.test.ts', 'w') as f:
    f.write(test_code)

print("✓ Test file created: demo/tests/math.test.ts")

In [ ]:
# Run the agent — simple mode (default)
# --simple 3  →  try up to 3 Artisan iterations
# --no-escalate  →  don't fall back to full mode (keep it simple for this demo)
!cd demo && npx ma-loop run src/math.ts \
    --test "npx vitest run" \
    --simple 3 \
    --no-escalate \
    --verbose

### What you'll see — one iteration at a time

Each simple mode iteration prints:

```
============================================================
Simple Mode: up to 3 iteration(s)
============================================================

[Simple 1/3]
Simple Mode: Artisan generating code...
  Artisan: Changed multiply() operator from + to * to compute the product correctly
  Change:  line 11: return a + b → return a * b
Simple Mode: Running tests...
  Tests: ✓ ALL PASSED (3 tests)
  Cost:  $0.0031 this iteration

Simple Mode: Solved in 1/3 iterations
============================================================
Status:     SUCCESS
Mode:       Simple only
Iterations: 1 simple / 0 full
Cost:       $0.003 simple / $0.000 full / $0.003 total
Duration:   4.2s
```

**Every iteration shows you:**
- What the Artisan reasoned / decided to change
- The exact line-level change
- Which tests passed or failed (with error messages on failure)
- Cost for that iteration

This is the same visibility you had in the original micro-agent, now in the fast simple mode.

## Part 5: What Happens When Simple Mode Fails — Auto-Escalation

Now let's use a bug that's **too complex for simple mode alone** — one that needs full codebase context to understand.

We won't simulate a real unsolvable bug here, but here's exactly what the output looks like when
simple mode exhausts its iterations and escalates to full mode:

```
============================================================
Simple Mode: up to 5 iteration(s)
============================================================

[Simple 1/5]
Simple Mode: Artisan generating code...
  Artisan: Added null check before property access in processOrder()
Simple Mode: Running tests...
  Tests: ✗ 2 failed / 8 total
    ✗ processOrder handles missing user
      Expected: { status: 'error', code: 'USER_NOT_FOUND' }
    ✗ processOrder validates inventory
      TypeError: Cannot read properties of undefined (reading 'stock')
  Cost:  $0.0041 this iteration

[Simple 2/5]
Simple Mode: Artisan generating code...
  Artisan: Added inventory stock check before decrement
Simple Mode: Running tests...
  Tests: ✗ 2 failed / 8 total
    ✗ processOrder handles missing user
      Expected: { status: 'error', code: 'USER_NOT_FOUND' }
    ✗ processOrder validates inventory
      TypeError: Cannot read properties of undefined (reading 'stock')
  Cost:  $0.0039 this iteration

... (5 iterations, all failing with same errors)

============================================================
Escalating to Full Mode after 5 simple iteration(s)
   Summary: SIMPLE MODE HISTORY (5 iterations, all failed):
   Iteration 1: Added null check before property access. Tests: Expected ...
   Remaining budget: $1.975
============================================================

Full Mode: up to 25 iteration(s) remaining

[Full 1/25]
Phase 1: Librarian analyzing context...
  [Librarian receives: PRIOR ATTEMPTS:\nSIMPLE MODE HISTORY (5 iterations, all failed)...]
Phase 2: Artisan generating code...
  Artisan: Librarian revealed that inventory.stock lookup goes through
           InventoryService which lazy-loads — user must exist first.
           Reordering validation: user check → then inventory check.
Phase 3: Critic reviewing code...
Phase 4: Running tests...
  Tests: ✓ ALL PASSED (8 tests)

Full Mode: Solved in 2 additional iterations
============================================================
Status:     SUCCESS
Mode:       Simple -> Full (escalated)
Iterations: 5 simple / 2 full / 7 total
Cost:       $0.021 simple / $0.089 full / $0.110 total
Duration:   52.3s
```

**Notice:** The Librarian receives the full failure history from simple mode.
It knows exactly what was already tried. It can spot patterns simple mode couldn't — in this case,
that `InventoryService` lazy-loads through a dependency that requires user existence first.
Simple mode didn't know that. Librarian + full codebase context does.

## Part 6: All the CLI Flags


In [ ]:
# See all run command flags
!npx ma-loop run --help

### Mode flags

```bash
# Default: simple mode, 5 iterations, auto-escalate if needed
ma-loop run src/math.ts

# Simple mode with custom iteration count
ma-loop run src/math.ts --simple 3

# Simple mode only — exit with failure instead of escalating
ma-loop run src/math.ts --simple 5 --no-escalate

# Full mode from the start (original pre-002 behaviour)
ma-loop run src/math.ts --full
```

### Budget flags

```bash
# Max 50 cents total across both phases
ma-loop run src/math.ts --max-budget 0.50

# Cap at 10 minutes wall time
ma-loop run src/math.ts --max-duration 10

# Cap total iterations across both phases
ma-loop run src/math.ts --max-iterations 20
```

### Agent model flags

```bash
# Override which model each agent uses
ma-loop run src/math.ts \
    --librarian gemini-2.0-flash \
    --artisan claude-opus-4.6 \
    --critic gpt-4.1
```

### Test command flag

```bash
# Specify how to run tests (default: npm test)
ma-loop run src/math.ts --test "npx vitest run --reporter verbose"
ma-loop run src/math.ts --test "pytest tests/ -v"
ma-loop run src/math.ts --test "cargo test"
```

## Part 7: How the Loop Works Internally

This is what happens inside `ma-loop run` every iteration:

```
┌─────────────────────────────────────────────────────────────────────┐
│                         PHASE A: Simple Mode                        │
│                                                                     │
│  iteration 1                                                        │
│  ┌────────────────────────────────────────────────────────────┐     │
│  │  Fresh AgentContext (no history carried over)              │     │
│  │        ↓                                                   │     │
│  │  Artisan  ←  objective + target file + last test failure   │     │
│  │        ↓ writes code change                                │     │
│  │  Tests    ←  runs your test command                        │     │
│  │        ↓                                                   │     │
│  │  Pass? → EXIT (success)                                    │     │
│  │  Fail? → save SimpleIterationRecord → next iteration       │     │
│  └────────────────────────────────────────────────────────────┘     │
│                                                                     │
│  (repeat up to N times)                                             │
└─────────────────────────────────────────────────────────────────────┘
            │
            │  (if all N iterations failed)
            ▼
┌─────────────────────────────────────────────────────────────────────┐
│                      PHASE B: Escalation Gate                       │
│                                                                     │
│  buildFailureSummary(records[])                                     │
│    → naturalLanguageSummary: "Iteration 1: changed X. Tests: err"  │
│    → uniqueErrorSignatures: ["Expected 12, received NaN"]           │
│    → (capped at 2000 chars / ~500 tokens)                           │
│                                                                     │
│  withEscalationContext(context, summary)                            │
│    → context.escalationContext = naturalLanguageSummary             │
│    → (immutable — original context untouched)                       │
│                                                                     │
│  Budget check: enough left to run full mode?                        │
│  --no-escalate? → skip to exit (failure)                            │
└─────────────────────────────────────────────────────────────────────┘
            │
            ▼
┌─────────────────────────────────────────────────────────────────────┐
│                       PHASE C: Full Mode                            │
│                                                                     │
│  Librarian ← objective + codebase + PRIOR ATTEMPTS: [summary]      │
│         ↓  ranked files + context summary (informed by history)     │
│  Artisan  ← librarian context + last test failure                  │
│         ↓  writes code change                                       │
│  Critic   ← reviews code change                                     │
│         ↓  approved?                                                │
│  Tests    ← runs your test command                                  │
│         ↓                                                           │
│  Pass? → EXIT (success)                                             │
│  Fail? → next full mode iteration                                   │
└─────────────────────────────────────────────────────────────────────┘
```

### The Gold Standard: Fresh Context Every Iteration

Every LLM call in every iteration starts with a **fresh message array**:

```typescript
messages: [
  { role: 'system', content: systemPrompt },
  { role: 'user',   content: buildPrompt(context) }
]
```

There is **no growing conversation history**. Iteration 10 doesn't carry baggage from iterations 1–9.
This is intentional — it prevents the LLM from anchoring on previous mistakes and keeps token costs flat
whether you run 3 iterations or 60.

## Part 8: Python Project Example

Works exactly the same for Python — just point it at a `.py` file and tell it your test command.

In [ ]:
import os

os.makedirs('demo_py', exist_ok=True)

# Broken Python implementation
broken_py = '''
def calculate_discount(price: float, percent: float) -> float:
    """Apply a percentage discount to a price."""
    # BUG: divides instead of subtracts
    return price / (percent / 100)
'''

with open('demo_py/calculator.py', 'w') as f:
    f.write(broken_py)

# Test for it
test_py = '''
from calculator import calculate_discount

def test_ten_percent_discount():
    assert calculate_discount(100.0, 10) == 90.0

def test_fifty_percent_discount():
    assert calculate_discount(200.0, 50) == 100.0
'''

with open('demo_py/test_calculator.py', 'w') as f:
    f.write(test_py)

print("✓ Python demo created")

In [ ]:
# Run against a Python file
!cd demo_py && npx ma-loop run calculator.py \
    --test "pytest test_calculator.py -v" \
    --simple 3 \
    --no-escalate

> See the full Python tutorial: [`docs/tutorials/python.md`](../tutorials/python.md)

## Part 9: Using an Objective Instead of a File

You don't have to point at a specific file. You can describe what you want built.

In [ ]:
# Run with a natural language objective
!npx ma-loop run \
    --objective "Implement a rate limiter class that allows N requests per second and blocks when the limit is exceeded" \
    --test "npm test" \
    --simple 5

In this mode, the Artisan generates code from scratch based on the objective,
and the loop continues until the tests you wrote pass.

## Part 10: Budget Management

Budget is **shared across both phases** — simple mode and full mode draw from the same pool.

In [ ]:
# Tight budget — stop if total spend hits $0.10
!npx ma-loop run src/math.ts \
    --max-budget 0.10 \
    --max-duration 5 \
    --simple 3

If budget runs out during simple mode:
```
[Simple 2/3]
Budget exceeded during simple mode, stopping

Status:     FAILED
Mode:       Simple only
Iterations: 2 simple / 0 full
Cost:       $0.099 simple / $0.000 full / $0.099 total
```

Budget exhaustion also **blocks escalation** — the system won't start full mode if there's not enough budget left.

### Cost ballpark

| Mode | Typical per-iteration cost |
|---|---|
| Simple (Artisan only) | ~$0.003 – $0.008 |
| Full (Librarian + Artisan + Critic) | ~$0.015 – $0.060 |

For most simple bugs, **1–3 simple mode iterations** solve the problem for under $0.02 total.

## Part 11: Configuration File

You can store defaults in `.micro-agent.yml` instead of passing flags every time.

In [ ]:
config_content = """
# .micro-agent.yml
models:
  librarian:
    provider: google
    model: gemini-2.0-pro
    temperature: 0.3
  artisan:
    provider: anthropic
    model: claude-sonnet-4-5-20250929
    temperature: 0.7
  critic:
    provider: openai
    model: gpt-4.1-mini
    temperature: 0.2

testing:
  defaultCommand: npm test
  framework: vitest

budget:
  maxCostUsd: 2.00
  maxDurationMinutes: 15
  maxIterations: 30

memory:
  contextResetFrequency: 1   # Fresh context every iteration (gold standard)
"""

with open('.micro-agent.yml', 'w') as f:
    f.write(config_content)

print("✓ .micro-agent.yml written")

> See [`docs/tutorials/model-configuration.md`](../tutorials/model-configuration.md) for the full config reference.

## Part 12: Real-World Decision Tree

Here's how to pick the right flags for your situation:

```
Is the bug simple? (wrong operator, off-by-one, typo)
    YES → ma-loop run src/file.ts          (default: simple 5, auto-escalate)
    YES, fast → ma-loop run src/file.ts --simple 3 --no-escalate

Does the fix need to understand how files relate?
    YES → ma-loop run src/file.ts --full

Are you on a tight budget?
    YES → ma-loop run src/file.ts --simple 3 --no-escalate --max-budget 0.05

Are you debugging something complex that simple mode already failed on?
    YES → ma-loop run src/file.ts --full   (skip straight to Librarian)

Running in CI with no budget for retries?
    YES → ma-loop run src/file.ts --simple 1 --no-escalate --max-budget 0.01
```

## Part 13: Reading the Output

Here's a complete annotated run:

```
🤖 Micro Agent starting (Ralph Loop engine)       ← startup

============================================================
Simple Mode: up to 5 iteration(s)                 ← Phase A begins
============================================================

[Simple 1/5]                                      ← iteration counter
Simple Mode: Artisan generating code...
  Artisan: Changed return a + b to return a * b   ← what artisan decided
  Change:  line 11: return a + b → return a * b   ← exact change
Simple Mode: Running tests...
  Tests: ✗ 1 failed / 3 total                     ← test summary
    ✗ multiply works                              ← which test failed
      Expected 12, received 7                     ← the error message
  Cost:  $0.0031 this iteration                   ← per-iteration cost

[Simple 2/5]
Simple Mode: Artisan generating code...
  Artisan: Corrected the multiply function operator
Simple Mode: Running tests...
  Tests: ✓ ALL PASSED (3 tests)                   ← success
  Cost:  $0.0028 this iteration

Simple Mode: Solved in 2/5 iterations             ← early exit!
============================================================
Status:     SUCCESS
Mode:       Simple only
Iterations: 2 simple / 0 full / 2 total
Cost:       $0.006 simple / $0.000 full / $0.006 total
Duration:   7.1s
```

## Part 14: Running the Test Suite

Micro-agent ships with 247 tests covering all its own logic.

In [ ]:
# Run the full test suite
!npm test

In [ ]:
# Run just the escalation-related tests
!npx vitest run tests/unit/lifecycle/simple-escalation.test.ts \
                 tests/integration/escalation-flow.test.ts \
                 --reporter verbose

## Further Reading

| What you want to learn | Where to go |
|---|---|
| TypeScript/JavaScript project setup | [`docs/tutorials/typescript-javascript.md`](../tutorials/typescript-javascript.md) |
| Python project setup | [`docs/tutorials/python.md`](../tutorials/python.md) |
| Rust project setup | [`docs/tutorials/rust.md`](../tutorials/rust.md) |
| Swapping models, using Ollama, per-agent config | [`docs/tutorials/model-configuration.md`](../tutorials/model-configuration.md) |
| Simple mode + escalation user guide | [`specs/002-simple-escalation/quickstart.md`](../../specs/002-simple-escalation/quickstart.md) |
| CLI flags contract (all flags documented) | [`specs/002-simple-escalation/contracts/cli-interface.md`](../../specs/002-simple-escalation/contracts/cli-interface.md) |
| Feature specification | [`specs/002-simple-escalation/spec.md`](../../specs/002-simple-escalation/spec.md) |
| Architecture decisions | [`specs/002-simple-escalation/research.md`](../../specs/002-simple-escalation/research.md) |
| Data model (types and state transitions) | [`specs/002-simple-escalation/data-model.md`](../../specs/002-simple-escalation/data-model.md) |

---

**Built on the Ralph Loop engine** — fresh context every iteration, shared budget across phases,
and an escalation bridge that means full mode never starts cold.